## 2. Nate 뉴스기사 제목 스크래핑하기

### 2-1 최신뉴스, 정치 , 경제, 사회, 세계, IT/과학   6개의 섹션의 뉴스를 출력하는 함수를 생성하여 스크래핑 하기 (필수)

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
from IPython.display import Image, display

section_dict = {
    100: "최신뉴스",
    200: "정치",
    300: "경제",
    400: "사회",
    500: "세계",
    600: "IT/과학",
}

def print_news(section_code):
    section_name = section_dict.get(section_code)
    
    if section_name is None:
        print('요청하신 키값이 없습니다.')
        return

    url = f'https://news.nate.com/recent?mid=n{section_code}'
    
    print(f'======> {section_name} <======')

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }

    res = requests.get(url, headers=headers)

    if res.ok:
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.text, 'html.parser')

        div_tag_list = soup.select('div.mlt01')

        for div_tag in div_tag_list:
            a_tag = div_tag.find('a')
            

            link = a_tag['href']
            title_tag = a_tag.select_one('span.ib')
            title = title_tag.text.strip() 

            print(title)
            print(link)

            # 이미지 처리
            img_tag = a_tag.find('img')
            if img_tag and img_tag.has_attr('src'):
                img_url = urljoin("https://news.nate.com", img_tag['src'])
                print(img_url)
                display(Image(url=img_url))
            
    else:
        print(f'에러코드 = {res.status_code}')


In [ ]:
print_news(100)  # 최신뉴스
print_news(200)  # 정치

### 2-2. 하나의 네이버 웹툰과 1개의 회차에 대한 Image 다운로드 하기 (필수)


In [33]:
import requests
import os
from bs4 import BeautifulSoup

def fetch_page(url, req_header):
    res = requests.get(url, headers=req_header)
    if res.ok:
        return res.text
    else:
        print(f'에러코드: {res.status_code}')
        return None

def get_img_url_list(html):
    soup = BeautifulSoup(html, 'html.parser')
    
    img_tags = soup.select("img[src*='IMAG01']")
    img_url_list = []
    for img_tag in img_tags:
        img_src = img_tag['src']
        img_url_list.append(img_src)
    return img_url_list

def create_img_dir(imgdir_name):
    os.makedirs(imgdir_name, exist_ok=True)

def download_imgs(img_url_list, imgdir_name, req_header):
    for img_url in img_url_list:
        res = requests.get(img_url, headers=req_header)
        if res.ok:
            img_data = res.content
            file_name = os.path.basename(img_url)
            file_path = os.path.join(imgdir_name, file_name)
            with open(file_path, 'wb') as file:
                print(f'Writing to {file_path}({len(img_data):,} bytes)')
                file.write(img_data)
        else:
            print(f'이미지 다운로드 실패: {img_url}, 상태코드: {res.status_code}')

def main(title, no, url):
    url = 'https://comic.naver.com/webtoon/detail?titleId=833255&no=44&week=tue'
    req_header = {
        'referer': 'url',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
    }
    imgdir_name = os.path.join('img', title, str(no))

    html = fetch_page(url, req_header)
    if html:
        img_url_list = get_img_url_list(html)
        create_img_dir(imgdir_name)
        download_imgs(img_url_list, imgdir_name, req_header)


In [34]:
main('낢이사는이야기', 48, 'https://comic.naver.com/webtoon/detail?titleId=833255&no=49&week=tue')

Writing to img\낢이사는이야기\48\20250211121545_dc7fce97ccfd61325bd1c797038e14e1_IMAG01_1.jpg(179,042 bytes)
Writing to img\낢이사는이야기\48\20250211121545_dc7fce97ccfd61325bd1c797038e14e1_IMAG01_2.jpg(124,150 bytes)
Writing to img\낢이사는이야기\48\20250211121545_dc7fce97ccfd61325bd1c797038e14e1_IMAG01_3.jpg(102,386 bytes)
Writing to img\낢이사는이야기\48\20250211121545_dc7fce97ccfd61325bd1c797038e14e1_IMAG01_4.jpg(153,009 bytes)
Writing to img\낢이사는이야기\48\20250211121545_dc7fce97ccfd61325bd1c797038e14e1_IMAG01_5.jpg(191,207 bytes)
Writing to img\낢이사는이야기\48\20250211121545_dc7fce97ccfd61325bd1c797038e14e1_IMAG01_6.jpg(219,719 bytes)
Writing to img\낢이사는이야기\48\20250211121545_dc7fce97ccfd61325bd1c797038e14e1_IMAG01_7.jpg(83,635 bytes)
Writing to img\낢이사는이야기\48\20250211121545_dc7fce97ccfd61325bd1c797038e14e1_IMAG01_8.jpg(170,745 bytes)
Writing to img\낢이사는이야기\48\20250211121545_dc7fce97ccfd61325bd1c797038e14e1_IMAG01_9.jpg(168,728 bytes)
Writing to img\낢이사는이야기\48\20250211121545_dc7fce97ccfd61325bd1c797038e14e1_IMAG01_10

### 2-3. 하나의 네이버 웹툰과 여러개의 회차에 대한 Image 다운로드 하기 (선택)


## 3. unsplash Image 다운로드

In [26]:
%pip install -q selenium webdriver-manager

Note: you may need to restart the kernel to use updated packages.


In [29]:
import os
import time
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

# 1. 이미지 저장 폴더 생성
save_folder = 'unsplash_img'
os.makedirs(save_folder, exist_ok=True)

# 2. Selenium WebDriver 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 브라우저 GUI 없이 실행
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# 3. Unsplash 'cat' 이미지 검색 페이지 접속
url = 'https://unsplash.com/s/photos/cat'
driver.get(url)
time.sleep(3)  # 페이지 로딩 대기

# 4. 이미지 태그 찾기
images = driver.find_elements(By.CSS_SELECTOR, 'img[srcset]')

print(f"이미지 {len(images)}개를 찾았습니다.")

# 5. 이미지 다운로드 (10개 정도 제한)
count = 0
for img in images:
    src = img.get_attribute('src')
    if src and "images.unsplash.com" in src:
        try:
            img_data = requests.get(src).content
            with open(os.path.join(save_folder, f"cat_{count+1}.jpg"), 'wb') as f:
                f.write(img_data)
            print(f"{count+1}번째 이미지 저장 완료.")
            count += 1
            if count >= 10:
                break
        except Exception as e:
            print(f"이미지 저장 실패: {e}")

# 6. 브라우저 종료
driver.quit()
print("모든 작업이 완료되었습니다.")


이미지 40개를 찾았습니다.
1번째 이미지 저장 완료.
2번째 이미지 저장 완료.
3번째 이미지 저장 완료.
4번째 이미지 저장 완료.
5번째 이미지 저장 완료.
6번째 이미지 저장 완료.
7번째 이미지 저장 완료.
8번째 이미지 저장 완료.
9번째 이미지 저장 완료.
10번째 이미지 저장 완료.
모든 작업이 완료되었습니다.
